In [1]:
# import the required libraries
import os
import ctools

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import SymLogNorm, LogNorm
from astropy import units as u

import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# for modify xml file of model.xml
from lxml import etree

In [3]:
import pandas as pd
from tqdm import tqdm

## **Parameters**

### **Common parameters**

In [4]:
pointing = np.array([83.6331, 22.5145])
fov = 5 
roi = 5
time = (0, 100)

In [5]:
no_crab_skymap = './data/dataset/no_crab_sky.fits'

### **Group 2 parameters**

In [6]:
energy=(0.1, 50)
irf = 'South_z20_0.5h'

## **Define folders**

In [7]:
# base folder of the datas
base_folder = './data/dataset'
if not os.path.isdir(base_folder):
    os.mkdir(base_folder)

# folder of observations
obs_folder = './data/dataset/observetions'
if not os.path.isdir(obs_folder):
    os.mkdir(obs_folder)

# folder of skymaps
skymaps_folder = './data/dataset/skymaps'
if not os.path.isdir(skymaps_folder):
    os.mkdir(skymaps_folder)
    
# base folder of the datas
tmp_folder = './tmp'
if not os.path.isdir(tmp_folder):
    os.mkdir(tmp_folder)

## **Create dataset**

In [8]:
n = 1000
max_deg = 3
p_no_crab = 0.4
p_1_crab = 0.8
p_2_crab = 0.9
p_3_crab = 1

In [9]:
X = []
y = []
num_sorces = []
file_names = []
offset = []
for i in range(n):
    eps = np.random.uniform()
    if eps <= p_no_crab:
        X.append([0, 0])
        y.append('no')
        num_sorces.append(0)
        file_names.append('crab_sky'+ "%02d" %(i,) +'.fits')
    elif eps <= p_1_crab:
        X.append([np.random.uniform(pointing[0]-max_deg, pointing[0]+max_deg),
                  np.random.uniform(pointing[1]-max_deg, pointing[1]+max_deg)])
        y.append('yes')
        num_sorces.append(1)
        file_names.append('crab_sky'+ "%02d" %(i,) +'.fits')
    elif eps <= p_2_crab:
        for j in range(2):
            X.append([np.random.uniform(pointing[0]-max_deg, pointing[0]+max_deg),
                      np.random.uniform(pointing[1]-max_deg, pointing[1]+max_deg)])
            y.append('yes')
            num_sorces.append(2)
            file_names.append('crab_sky'+ "%02d" %(i,) +'.fits')
    elif eps <= p_3_crab:
        for j in range(3):
            X.append([np.random.uniform(pointing[0]-max_deg, pointing[0]+max_deg),
                      np.random.uniform(pointing[1]-max_deg, pointing[1]+max_deg)])
            y.append('yes')
            num_sorces.append(3)
            file_names.append('crab_sky'+ "%02d" %(i,) +'.fits')

In [10]:
df = pd.DataFrame({'File_name':file_names,
                   'Source-ra':np.array(X)[:,0],
                   'Source-dec':np.array(X)[:,1],
                   'N_sources':num_sorces,
                   'Label': y})
df

,File_name,Source-ra,Source-dec,N_sources,Label
0,crab_sky00.fits,0.000000,0.000000,0,no
1,crab_sky01.fits,0.000000,0.000000,0,no
2,crab_sky02.fits,82.414278,24.671857,1,yes
3,crab_sky03.fits,0.000000,0.000000,0,no
4,crab_sky04.fits,81.586806,24.264234,1,yes
...,...,...,...,...,...
1266,crab_sky996.fits,80.933802,21.758928,3,yes
1267,crab_sky996.fits,83.727460,20.193317,3,yes
1268,crab_sky997.fits,82.221058,25.419590,1,yes
1269,crab_sky998.fits,0.000000,0.000000,0,no


## **Functions**

### ctobssim

In [11]:
def run_ctobssim(model, pointing, output, energy=(0.03, 150), time=(0, 1200), fov=5, 
                 caldb='prod3b-v2', irf='South_z40_0.5h', seed=1):
    sim = ctools.ctobssim()
    sim["inmodel"] = model
    sim["outevents"] = output
    sim["caldb"] = caldb
    sim["irf"] = irf
    sim["ra"] = pointing[0]
    sim["dec"] = pointing[1]
    sim["rad"] = fov
    sim["tmin"] = time[0]
    sim["tmax"] = time[1]
    sim["emin"] = energy[0]
    sim["emax"] = energy[1]
    sim["seed"] = seed
    sim.execute()

### skymap

In [12]:
def run_skymap(obs, output, energy=(0.03, 150), roi=5, caldb='prod3b-v2', 
               irf='South_z40_0.5h', wbin=0.02):
    nbin = int(roi*2/wbin)
    skymap = ctools.ctskymap()
    skymap['inobs'] = obs
    skymap['outmap'] = output
    skymap['irf'] = irf
    skymap['caldb'] = caldb
    skymap['emin'] = energy[0]
    skymap['emax'] = energy[1]
    skymap['usepnt'] = True
    skymap['nxpix'] = nbin
    skymap['nypix'] = nbin
    skymap['binsz'] = wbin
    skymap['bkgsubtract'] = 'IRF'
    skymap.execute()

### generateSourceModel

In [13]:
def generateSourceModel(source, n_sources):
    if n_sources == 0:
        model = './models/no_crab.xml'
        return model
    elif n_sources == 1:
        model = './models/crab1.xml'
    elif n_sources == 2:
        model = './models/crab2.xml'
    elif n_sources == 3:
        model = './models/crab3.xml'
    tree = etree.parse(model)
    root = tree.getroot()
    # change value of spatialmodel source:
    for i in range(len(source)):
        spatialmodel = root[i][1]
        # ra value
        spatialmodel[0].set('value', str(source[i,0]))
        # dec value
        spatialmodel[1].set('value', str(source[i,1]))
    # Rewrite new values on the xml file
    tree.write(model, pretty_print=True)
    return model

### createSource

In [14]:
def createSource(idx_obs,
                 model,
                 pointing=(83.6331, 22.5145),
                 energy=(0.03, 150),
                 fov=5,
                 roi=5,
                 irf='South_z20_0.5h',
                 time=(0, 1200),
                 data_folder='./data/dataset', 
                 data_folder_sim=None, 
                 data_folder_skymap=None,
                 seed=1,
                 print_name_file=False):

    # create the simulation
    if not data_folder_sim == None:
        folder = data_folder_sim
    else:
        folder = data_folder
        
    # I can create the observetion
    obs=folder+'/observetion'+str(idx_obs)+'.fits'
    run_ctobssim(model=model, pointing=pointing, energy=energy, 
                 time=time, fov=fov, irf=irf, output=obs, seed=seed)
    if print_name_file:
        print(model)
        print(obs)
    # ... and create skymap
    if not data_folder_skymap == None:
        folder = data_folder_skymap
    else:
        folder = data_folder
    skymap=folder+'/crab_sky'+str(idx_obs)+'.fits'
    if print_name_file:
        print(skymap)
    run_skymap(obs=obs, output=skymap, energy=energy, irf=irf)
    

### genSimulations

In [15]:
def genSimulations(df, 
                   pointing=(83.6331, 22.5145),
                   energy=(0.03, 150),
                   fov=5,
                   roi=5,
                   irf='South_z20_0.5h',
                   time=(0, 1200),
                   data_folder='./data/dataset', 
                   model='./models/crab1.xml', 
                   data_folder_sim=None,
                   data_folder_skymap=None,
                   seed = 1,
                   skymap_seed_ub=2048): # upperbound skymap random seed
    
    filename = ''
    k = 0
#     np.random.seed(seed=1)
    np.random.seed(seed=seed)
    for i in tqdm(range(len(df))):
        if filename == df.iloc[i]['File_name']:
            continue
        filename = df.iloc[i]['File_name']
        
        num_sources = df.iloc[i]['N_sources']
        if num_sources == 0: 
            X = np.array([0, 0])
        else:
            X = []
            for j in range(i, i+num_sources):
                X.append([df.iloc[j]['Source-ra'],
                          df.iloc[j]['Source-dec']])
            X = np.array(X)
        # overwrite the previous model with the new source
        model = generateSourceModel(X, num_sources)
        
        # create the seed of the skymap
        skymap_seed = np.random.randint(skymap_seed_ub)
        
        if (not data_folder_sim==None) and (not data_folder_skymap==None):
            createSource(k, model, energy=energy, time=time, 
                         fov=fov, roi=roi, irf=irf, 
                         data_folder_sim=data_folder_sim, 
                         data_folder_skymap=data_folder_skymap,
                         seed=skymap_seed)
        else:
            createSource(k, model, energy=energy, time=time, fov=fov, 
                         roi=roi, irf=irf, data_folder=data_folder,
                         seed=skymap_seed)
        k += 1

In [16]:
genSimulations(df,
               data_folder_sim=obs_folder, 
               data_folder_skymap=skymaps_folder)

100%|██████████| 1271/1271 [6:21:31<00:00, 18.01s/it]  


## **Save .csv**

In [17]:
df.to_csv(base_folder+'/skymap_set.csv', index=False)

In [18]:
df

,File_name,Source-ra,Source-dec,N_sources,Label
0,crab_sky00.fits,0.000000,0.000000,0,no
1,crab_sky01.fits,0.000000,0.000000,0,no
2,crab_sky02.fits,82.414278,24.671857,1,yes
3,crab_sky03.fits,0.000000,0.000000,0,no
4,crab_sky04.fits,81.586806,24.264234,1,yes
...,...,...,...,...,...
1266,crab_sky996.fits,80.933802,21.758928,3,yes
1267,crab_sky996.fits,83.727460,20.193317,3,yes
1268,crab_sky997.fits,82.221058,25.419590,1,yes
1269,crab_sky998.fits,0.000000,0.000000,0,no
